In [ ]:
import pyodbc
import smtplib
import pandas as pd
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Configuration
server = ''
database = ''
username = ''
password = ''
driver = '{ODBC Driver 17 for SQL Server}'

smtp_server = 'smtp.gmail.com'
smtp_port = '587'
email_user = ''
email_password = ''

# set_email = 'scsit.db3@sparecare.in;hanish.khattar@sparecare.in;manish.sharma@sparecare.in'

# --- 1. Send Email Function ---
def send_email(to_email, cc_email, subject, body):
    try:
        msg = MIMEMultipart()
        msg['From'] = email_user
        msg['To'] = to_email
        # msg['To'] = 'scsit.db3@sparecare.in,vishu23.offical@gmail.com,mushtakeemahmad11@gmail.com'
        msg['Cc'] = cc_email
        # msg['Cc'] = ''
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html'))

        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(email_user, email_password)
            server.send_message(msg)

        print(f"Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"Email error: {e}")
        return False

# --- 2. Main ---
def main():
    try:
        conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}')
        cursor = conn.cursor()

        cursor.execute("exec Seller_Action_Pending")
        conn.commit()

        df = pd.read_sql_query("SELECT * FROM INCORRECT_INVOICE WHERE Mail_sent IS NULL", conn)

        if df.empty:
            print("No data to process.")
            return

        unique_sellers = df['Seller'].unique()

        for seller in unique_sellers:
            sd = df[df['Seller'] == seller]

            table_rows = ""
            for _, row in sd.iterrows():
                table_rows += f"""
                <tr>
                    <td style="padding: 5px">{row['Seller']}</td>
                    <td style="padding: 5px">{row['Buyer']}</td>
                    <td style="padding: 5px">{row['DispatchOrderNo']}</td>
                    <td style="padding: 5px">{row['LRNumber']}</td>
                    <td style="padding: 5px">{row['LRDate']}</td>
                    <td style="padding: 5px">{row['Value']}</td>
                    <td style="padding: 5px">{row['Reason']}</td>
                </tr>
                """

            table_html = f"""
            <table border="1" style="border-collapse: collapse;" >
                <tr style="background-color: #E0E0E0;">
                    <th>Seller</th>
                    <th>Buyer</th>
                    <th>Dispatch Order_No</th>
                    <th>LR/AWB Number</th>
                    <th>LR/AWB Date</th>
                    <th>Invoice Value (in Rs)</th>
                    <th>Reason</th>
                </tr>
                {table_rows}
            </table>
            """
            first_row = sd.iloc[0]
            to_email = first_row['ToEmail']
            cc_email = first_row['ccemail']


            # first_row = sd.iloc[0]
            # to_email = row['ToEmail']
            # cc_email = f"{row['ccemail']}"
            subject = f"Incorrect Invoice Updated in Gainer - {seller}"

            body = f"""
           <html>
        <body>
            <p>Dear Sir,</p>
            <p>Kindly note Sales Realisation of following transactions is put on hold due to updation of <strong>"Incorrect / Performa Invoice"</strong> in Gainer Portal</p>
            {table_html}
            <p>It is requested to kindly upload the correct TAX INVOICE (IRN Invoice) against above mentioned Dispatch Orders / LR number.</p>
            <p>Path for correct Invoice upload:<br>
            Spare Care Gainer > <strong>Seller Action Pending</strong></p>
            <p>In case of any support, pl revert.</p>
            <p>Thanks & Regards<br>
            Team Gainer</p>
        </body>
    </html>
    """

            if send_email(to_email, cc_email, subject, body):
                try:
                    dispatch_ids = tuple(sd['DispatchOrderNo'].unique())
                    if len(dispatch_ids) == 1:
                        dispatch_ids = f"('{dispatch_ids[0]}')"
                    else:
                        dispatch_ids = str(dispatch_ids)
                    cursor.execute(f"""
                        UPDATE INCORRECT_INVOICE 
                        SET Mail_sent = 1, updatedOn = GETDATE() 
                        WHERE DispatchOrderNo IN {dispatch_ids}
                    """)
                    conn.commit()
                except Exception as e:
                    print(f"Error updating for seller {seller}: {e}")

    except Exception as ex:
        print(f"Database error: {ex}")
    finally:
        if 'conn' in locals():
            conn.close()

if __name__ == "__main__":
    main()
